In [ ]:
import pandas as pd
from os.path import join as path_join
import joblib
import pickle

import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVR

import matplotlib as mpl
import matplotlib.pyplot as plt

## get data (mnist)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from sklearn.datasets import fetch_openml
# mnist = fetch_openml('mnist_784', version=1)
# mnist.keys()

In [ ]:
# X, y = mnist["data"], mnist["target"]

In [ ]:
# folder_name = "/content/drive/My Drive/HandsonML/"
folder_name = ""

In [ ]:
# with open(folder_name + 'mnist784.pickle', 'wb') as f:
#     pickle.dump((X,y), f)

In [ ]:
with open(folder_name + 'mnist784.pickle', 'rb') as f:
    X,y = pickle.load(f)

In [ ]:
X.shape, y.shape

In [ ]:
y[0]

In [ ]:
some_digit = X[0:1].values
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()

In [ ]:
y = y.astype(np.uint8)

In [ ]:
# sets are already shuffled
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [ ]:
# from sklearn.base import clone
# from sklearn.linear_model import SGDClassifier
# from sklearn.model_selection import StratifiedKFold
# import numpy as np

# skfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# for train_index, test_index in skfolds.split(X_train, y_train_5):
#     clone_clf = clone(sgd_clf)

#     X_train_folds = X_train.iloc[train_index]
#     y_train_folds = y_train_5.iloc[train_index]
#     X_test_fold = X_train.iloc[test_index]
#     y_test_fold = y_train_5.iloc[test_index]

#     clone_clf.fit(X_train_folds, y_train_folds)
#     y_pred = clone_clf.predict(X_test_fold)
#     n_correct = np.sum(y_pred == y_test_fold)
#     print(n_correct / len(y_pred))  # prints the accuracy for each fold


## task1 logreg


In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# cross_val_score(logreg, X_train, y_train, cv=3, scoring="accuracy")
[0.91275, 0.91565, 0.9171 ]

In [ ]:
# logreg = LogisticRegression(max_iter=1000)

# pipeline_for_scaler = Pipeline(steps=[
#     ('scaler', StandardScaler()),  # Use StandardScaler or MinMaxScaler
#     ('classifier', logreg)
# ])

# param_grid = {
#     'scaler': [StandardScaler(), MinMaxScaler()],
# }

In [ ]:
# grid_search = GridSearchCV(pipeline_for_scaler, param_grid, cv=3, scoring='accuracy')

# # Fit GridSearchCV or RandomizedSearchCV
# grid_search.fit(X_train, y_train)

In [ ]:
# best_scaler = grid_search.best_estimator_.steps[0][1]
best_scaler = MinMaxScaler()

In [ ]:
# # Feature engineering
# feature_engineering = Pipeline(steps=[
#     ('poly', PolynomialFeatures(degree=2, include_bias=False))
# ])

# Logistic Regression model
logreg = LogisticRegression(max_iter=1000)

# Complete pipeline
pipeline = Pipeline(steps=[
    ('scaler', best_scaler),  # Use StandardScaler or MinMaxScaler
    # ('feature_engineering', feature_engineering),
    ('classifier', logreg)
])

# Define parameter grid for GridSearchCV or RandomizedSearchCV
param_grid = {
    'classifier__solver': ['liblinear', 'saga', 'newton-cg', 'lbfgs'],
    'classifier__max_iter': [1000, 2000, 3000],
    # 'feature_engineering__poly__degree': [1, 2, 3]
}

time versus result:

gridsearch: dohuya

randomsearch 60 iter 280 min, 12 iter 180 min (same res),  

bayesian - same res and fast

halving grid search

halving random search - shit 


In [ ]:
from skopt import BayesSearchCV

# Define parameter grid for Bayesian optimization
param_grid_bayes = {
    'classifier__solver': ['liblinear', 'saga', 'newton-cg', 'lbfgs'],
    'classifier__max_iter': [1000, 2000, 3000],
    # 'feature_engineering__poly__degree': [1, 2, 3]
}

In [ ]:
# Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_grid_bayes,
    n_iter=8,
    cv=3,
    scoring='accuracy',
    random_state=42,
    verbose=3
)

# Fit the model
bayes_search.fit(X_train, y_train)

print("Best parameters (BayesSearchCV):", bayes_search.best_params_)
print("Best score (BayesSearchCV):", bayes_search.best_score_)


In [ ]:
y_train.value_counts()
# no imbalance

# one vs one

In [157]:
def get_accuracy(model, X, y_true):
    y_pred = model.predict(X)
    n_correct = np.sum(y_pred == y_true)
    accuracy = n_correct / len(y_pred)
    return accuracy

In [96]:
from sklearn.multiclass import OneVsOneClassifier

In [97]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Logistic Regression model with One-vs-One strategy
logreg_1v1 = OneVsOneClassifier(LogisticRegression())

# Complete pipeline
pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),  # Use StandardScaler or MinMaxScaler
    ('classifier', logreg_1v1)
])

param_grid = {
    'classifier__estimator__solver': ['liblinear'],
    'classifier__estimator__max_iter': [500, 3000, 5000, 10000, 15000]
}

In [ ]:
# Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_grid,
    n_iter=8,
    cv=3,
    scoring='accuracy',
    random_state=42,
    verbose=3
)

# Fit the model
bayes_search.fit(X_train, y_train)

print("Best parameters (BayesSearchCV):", bayes_search.best_params_)
print("Best score (BayesSearchCV):", bayes_search.best_score_)

### feature importance

logreg dosn't have feature_importances but it has coef_ so we'll just skip this part. bat this can be improved. 

there are other ways to find more valuable features


### feature enginiering

In [ ]:
import cv2

def contour_has_circle(row, circularity_threshold=0.7):
    image = np.array(row)
    img = image.reshape(28, 28).astype(np.uint8)
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity > circularity_threshold:
            return 1
    return 0

In [55]:
def check_circles_in_digits(X, y, circle_detection_func, target_digits=None):
    if target_digits is None:
        target_digits = y.unique()
    results = {}
    
    for digit in target_digits:
        # Get indices of the current digit
        digit_indices = np.where(y.astype(int) == digit)[0]
        
        # Get the images for this digit
        X_digit_images = X.iloc[digit_indices]
        # Apply the provided circle detection function
        has_circles = X_digit_images.apply(circle_detection_func, axis=1)
        # Calculate statistics
        total_count = len(digit_indices)
        circle_count = np.sum(has_circles)
        circle_percentage = (circle_count / total_count) * 100
        
        results[digit] = {
            'total': total_count,
            'with_circle': circle_count,
            'percentage': circle_percentage
        }
    
    return results

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def train_circle_detector(X, y):
    # Manually label some digits as having circles
    circle_labels = np.isin(y.astype(int), [0, 6, 8, 9]).astype(int)
    
    clf = Pipeline([
        ('scaler', MinMaxScaler()),
        ('classifier', LogisticRegression(max_iter=3000))
    ])
    clf.fit(X, circle_labels)
    
    return clf

circle_detector = train_circle_detector(X_train, y_train)

In [102]:
def ml_has_circle(image):
    return circle_detector.predict(image.to_frame().T)[0]

In [56]:
circle_result_counter = check_circles_in_digits(X_train, y_train, contour_has_circle)
circle_result_counter

0        0
11       0
35       0
47       0
65       0
        ..
59956    0
59960    0
59968    0
59993    0
59997    0
Length: 5421, dtype: int64
1        1
21       1
34       0
37       1
51       1
        ..
59940    0
59944    0
59952    0
59972    0
59987    1
Length: 5923, dtype: int64
2        0
9        0
20       0
26       0
53       0
        ..
59933    0
59941    0
59943    0
59951    0
59975    0
Length: 5842, dtype: int64
3        0
6        0
8        0
14       0
23       0
        ..
59958    0
59965    0
59979    0
59984    0
59994    0
Length: 6742, dtype: int64
4        0
19       0
22       0
33       0
43       0
        ..
59948    0
59969    1
59973    0
59990    0
59992    0
Length: 5949, dtype: int64
5        0
16       0
25       0
28       0
76       0
        ..
59971    0
59974    0
59983    0
59985    0
59991    0
Length: 5958, dtype: int64
7        0
10       0
12       0
27       0
30       0
        ..
59961    0
59964    0
59978    0
59980    0
59

{5: {'total': 5421, 'with_circle': 21, 'percentage': 0.387382401770891},
 0: {'total': 5923, 'with_circle': 5044, 'percentage': 85.15954752659125},
 4: {'total': 5842, 'with_circle': 26, 'percentage': 0.4450530640191715},
 1: {'total': 6742, 'with_circle': 84, 'percentage': 1.2459210916641947},
 9: {'total': 5949, 'with_circle': 52, 'percentage': 0.8740964868045049},
 2: {'total': 5958, 'with_circle': 19, 'percentage': 0.3188989593823431},
 3: {'total': 6131, 'with_circle': 32, 'percentage': 0.5219376936878161},
 6: {'total': 5918, 'with_circle': 70, 'percentage': 1.1828320378506252},
 7: {'total': 6265, 'with_circle': 26, 'percentage': 0.41500399042298486},
 8: {'total': 5851, 'with_circle': 281, 'percentage': 4.802597846521962}}

In [54]:
circle_result_ml = check_circles_in_digits(X_train, y_train, ml_has_circle)
circle_result_ml

{5: {'total': 5421,
  'with_circle': array([1029]),
  'percentage': array([18.98173769])},
 0: {'total': 5923,
  'with_circle': array([5368]),
  'percentage': array([90.62974844])},
 4: {'total': 5842,
  'with_circle': array([859]),
  'percentage': array([14.70386854])},
 1: {'total': 6742,
  'with_circle': array([116]),
  'percentage': array([1.7205577])},
 9: {'total': 5949,
  'with_circle': array([4305]),
  'percentage': array([72.36510338])},
 2: {'total': 5958,
  'with_circle': array([649]),
  'percentage': array([10.89291709])},
 3: {'total': 6131,
  'with_circle': array([239]),
  'percentage': array([3.89822215])},
 6: {'total': 5918,
  'with_circle': array([5384]),
  'percentage': array([90.97668131])},
 7: {'total': 6265,
  'with_circle': array([449]),
  'percentage': array([7.16679968])},
 8: {'total': 5851,
  'with_circle': array([4259]),
  'percentage': array([72.7909759])}}

In [103]:
X_train_with_circle = X_train.copy()
X_train_with_circle['has_circle']  = X_train.apply(ml_has_circle, axis=1)

In [108]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('clf', OneVsOneClassifier(LogisticRegression(**{'max_iter': 5000, 'solver': 'liblinear'})))
])
cross_val_score(pipeline, X_train_with_circle, y_train, cv=3, scoring="accuracy")

array([0.9364, 0.9355, 0.9369])

In [109]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('clf', LogisticRegression(max_iter= 3000, solver= 'lbfgs'))
])
cross_val_score(pipeline, X_train_with_circle, y_train, cv=3, scoring="accuracy")

array([0.91655, 0.91655, 0.91905])

In [139]:
from sklearn.decomposition import PCA
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

log_reg = LogisticRegression(max_iter=3000, solver='lbfgs')
ovo_log_reg = OneVsOneClassifier(LogisticRegression(max_iter=5000, solver='liblinear'))

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA()),
    ('classifier', log_reg)
])

param_space = {
    'pca__n_components': Integer(50, 784),  # Explore PCA components from 50 to 784
}

In [145]:
search_pca_logreg = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=40,  # Number of parameter settings that are sampled
    scoring='accuracy',
    cv=3,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available cores
    verbose=3,
    random_state=42
)

In [146]:
search_pca_logreg.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=3,
              estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                        ('pca', PCA()),
                                        ('classifier',
                                         OneVsOneClassifier(estimator=LogisticRegression(max_iter=5000,
                                                                                         solver='liblinear')))]),
              n_iter=40, n_jobs=-1, random_state=42, scoring='accuracy',
              search_spaces={'pca__n_components': Integer(low=50, high=784, prior='uniform', transform='normalize')},
              verbose=3)

In [147]:
search_pca_logreg.best_score_

0.9371

In [142]:
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA()),
    ('classifier', ovo_log_reg)
])

In [143]:
search = BayesSearchCV(
    estimator=pipeline,
    search_spaces=param_space,
    n_iter=40,  # Number of parameter settings that are sampled
    scoring='accuracy',
    cv=3,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available cores
    verbose=3,
    random_state=42
)
search.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=3,
              estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                        ('pca', PCA()),
                                        ('classifier',
                                         OneVsOneClassifier(estimator=LogisticRegression(max_iter=5000,
                                                                                         solver='liblinear')))]),
              n_iter=40, n_jobs=-1, random_state=42, scoring='accuracy',
              search_spaces={'pca__n_components': Integer(low=50, high=784, prior='uniform', transform='normalize')},
              verbose=3)

In [148]:
search.best_estimator_, search.best_params_

(Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=180)),
                 ('classifier',
                  OneVsOneClassifier(estimator=LogisticRegression(max_iter=5000,
                                                                  solver='liblinear')))]),
 OrderedDict([('pca__n_components', 180)]))

### data augmentation

In [149]:
from scipy.ndimage import shift
def shift_digit(digit_array, dx, dy, new=0):
    return shift(digit_array.reshape(28, 28), [dy, dx], cval=new).reshape(784)

In [150]:
X_train_expanded = [X_train]
y_train_expanded = [y_train]
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    shifted_images = np.apply_along_axis(shift_digit, axis=1, arr=X_train, dx=dx, dy=dy)
    X_train_expanded.append(shifted_images)
    y_train_expanded.append(y_train)

X_train_expanded = np.concatenate(X_train_expanded)
y_train_expanded = np.concatenate(y_train_expanded)
X_train_expanded.shape, y_train_expanded.shape

((300000, 784), (300000,))

In [154]:
pipeline_final = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA(n_components=180)),
    ('classifier', ovo_log_reg)
])

In [155]:
pipeline_final.fit(X_train_expanded, y_train_expanded)

Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=180)),
                ('classifier',
                 OneVsOneClassifier(estimator=LogisticRegression(max_iter=5000,
                                                                 solver='liblinear')))])

In [158]:
get_accuracy(pipeline_final, X_test, y_test)

C:\Users\os_ta\PycharmProjects\Handsome_ML\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


0.946

In [161]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Define the pipeline
pipeline_final = Pipeline([
    ('scaler', MinMaxScaler()),
    ('pca', PCA(n_components=180)),
    ('classifier', OneVsOneClassifier(estimator=LogisticRegression(max_iter=5000)))
])

# Define the parameter search space with conditional parameters
param_search = {
    'classifier__estimator__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'classifier__estimator__solver': Categorical(['liblinear', 'saga']),
    'classifier__estimator__penalty': Categorical(['l1', 'l2']),
}

# Initialize BayesSearchCV with conditional parameters
bayes_search = BayesSearchCV(
    estimator=pipeline_final,
    search_spaces=param_search,
    n_iter=50,  # Number of iterations for searching
    cv=3,  # 5-fold cross-validation
    n_jobs=-1,  # Use all available CPU cores
    verbose=2,  # Set to 1 or 2 to see the progress
)

# Fit the search to your data (assuming X and y are your features and labels)
bayes_search.fit(X, y)

# Print the best parameters and the best score
print("Best Parameters: ", bayes_search.best_params_)
print("Best Score: ", bayes_search.best_score_)
    

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [ ]:
{'classifier__estimator__C': 0.3871210897414432, 'classifier__estimator__penalty': 'l2', 'classifier__estimator__solver': 'saga'}к